In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer,\
TrainingArguments, Trainer, DataCollatorForLanguageModeling

from datasets import load_dataset, Dataset

from functools import partial
import torch
import json
import evaluate

In [2]:
tokenizer = tokenizer = AutoTokenizer.from_pretrained("facebook/opt-125m", use_fast=False)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained("facebook/opt-125m")

In [21]:
with open("results/Ensemble/gpt2-xl-t5-3b-temp/2023-05-04-05-22-24-090143-fp16-0.3-1-squad-200/raw_data.json", "rb") as f:
    raw = json.load(f)
raw_data = Dataset.from_dict(raw)

In [22]:
preprocess_function = partial(preprocess, tokenizer = tokenizer)
raw_data = raw_data.map(preprocess_function,
            batched=True,
            num_proc=4,
            remove_columns=["sampled", "original"])
processed_data = raw_data.map(group_texts, batched=True, num_proc=4)

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

In [17]:
block_size = 128

def preprocess(examples, tokenizer):
    return tokenizer(examples["sampled"])

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [23]:
# For clm, set pad to eos and set mlm=False

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir="temp_opt",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    # weight_decay=0.01,
    # push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=processed_data,
    eval_dataset=processed_data,
    data_collator=data_collator,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [24]:
trainer.train()

***** Running training *****
  Num examples = 312
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 117
  Number of trainable parameters = 125239296


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 